### Newton's Method

Now that we know the *criteria* of a minimum for SSE, we need to *find* the $\hat\theta_0, \hat\theta_1$ that satisfy the criteria:

\\[
\begin{alignat*}{3}
\frac{\partial E}{\partial \hat\theta_0} &= \sum_{i=1}^N 2 \left(
                                              \left( \hat\theta_0 + \hat\theta_1 x_i \right) - y_i
                                            \right) \big(1\big) &&= 0\\
\frac{\partial E}{\partial \hat\theta_1} &= \sum_{i=1}^N 2 \left(
                                              \left( \hat\theta_0 + \hat\theta_1 x_i \right) - y_i
                                            \right) \big( x_i \big) &&= 0\\
\end{alignat*}
\\]

It turns out that we can use a little math to calculate exactly that $\hat\theta_0, \hat\theta_1$ are best. But I want to show you a way called *Newton's Method*. Newton's Method is super overkill for our *univariate* (one dimensional $x$ predictor variable) linear least squares regression problem.

However, once you learn Newton's Method, it is easy to learn a related method called *Gradient Descent* (abbreviated GD). GD is often used for *multiple* linear regression: when you have many dimensions to the predictor variable $x$. Gradient Descent is also the most common approach to training neural networks.

So what is Newton's Method? It is an algorithm for *finding zeroes of a function*. A *zero* of a function $f$ is any $x$ value where $f(x) = 0$. From the above criteria, we see that a choice of $\hat\theta_0, \hat\theta_1$ which is a zero for both partial derivative functions will be at a global minimum for the SSE error function.

### The Algorithm

Forget about the sum of squares error for a second. Let's just think about a function $f$, and we want to find an $x$ value where $f(x) = 0$.

Newton's method is an *iterative procedure* that starts with an initial estimate $x_0$ (or just a wild-ass guess) of where a function $f$ might satisfy $f(x_0) = 0$. Newton's method improves your guess iteratively to produce $x_1, x_2, \ldots$ until some $x_i$ satisfies $f(x_i) < \epsilon$. ($\epsilon$ is a common placeholder symbol for a number which is "as small as you want" but still non-zero.) Here is how:

1. Pick a random starting point $x_0$. Let $i = 0$.
2. Calculate $f(x_i)$ and $f'(x_i)$.
3. Set $x_{i+1}$ to $x_i - \frac{f(x_i)}{f'(x_i)}$.
4. If $f(x_{i + 1}) < \epsilon$, stop.
5. Set $i := i + 1$ and return to step 2.

Let's see it in code and then discuss how it works! I'm going to use an example function of $f(x) = 8(x - 2)^2 - 20$. Therefore $f'(x) = 16x$.

In [1]:
%matplotlib inline

from examples.newtons_method_example import NewtonsMethodExample

NewtonsMethodExample.run()

Let's talk about the concept of Newton's method. Let's assume $f$ is a line. It isn't in the video, but let's start slowly.

If you are at position $x_i$, then you have a (possibly negative) height of $f(x_i)$. Now, a line has a slope, which is $f'(x_i)$. For a line, the slope means that if you increase $x_i$ by one unit, you change $f(x_i)$ by $f'(x_i)$ units.

Now $f(x_i)$ is how far down you want to go. So how much should you move to the right? Well if $f(x_i) = 10$, then you want to change the $y$-value by $-f(x_i) = -10$ to get down to zero. How much must you increase $x_i$? Well, since you go up $f'(x_i)$ per unit increase in $x_i$, the answer is $-\frac{f(x_i)}{f'(x_i)}$.

Thus we have the math: $x_{i + 1} = x_i - \frac{f(x_i)}{f'(x_i)}$.

### Over and Undershooting

Now, when $f$ is a line our job is so easy. For non-line functions, the slope is always changing. $f'(x_i)$ is the slope at $x_i$, but it won't be the same for other values of $x_i$. So it is not strictly true that changing $x_i$ by $n$ units will change $f(x_i)$ by $nf'(x_i)$ units. This is what we are seeing in the parabola video above.

Look at the first step the algorithm makes. It predicts a very shallow slope, so it predicts that it will have to move quite far to reach the zero. But between $x_0$ and $x_1$, the slope of the parabola changes dramatically, so the parabola in reality reaches the zero much earlier than expected. The algorithm has overshot.

Likewise, the second step undershoots. The magnitude of the slope at $x_1$ is quite large, but the parabola slope is getting shallower. So when moving from $x_1$ to $x_2$, the parabola doesn't change as much as Newton's method thinks. That's why it comes up short.

All the same, after just a few steps Newton's method has basically perfectly approximated the zero. This is because a tangent line is a pretty decent approximation of a parabola over short distances.

For quadratic functions, Newton's method will overshoot when it is moving away from the minimum, because the slope is increasing in magnitude as you move away from the minimum value. But when Newton's method is moving toward the minimum, it will always undershoot, because the slope is decreasing in magnitude.

That means that for quadratic functions, Newton's method is always *guaranteed* to *converge* toward a zero. It may overshoot on the first step, but then it will consistently undershoot, always playing it safe and moving closer to the minimum.

For non-quadratic functions, the story is trickier. Newton's method isn't always guaranteed to converge to a stationary point. But we don't have to worry about that for now!

### Newton's Method for Optimization

As we said, since Newton's method finds zeros for a function, we can use it to find *critical points* (also called *stationary points*) for a function $f$ which are those $x$ where $f'(x) = 0$. If $f$ is differentiable, then maxima and minima always lie at critical points. One caveat: we'll see not all critical points are a maximum or a minimum.

Newton's method for optimization works too perfectly on a quadratic function of one variable. So let's apply it to a cubic function, to keep things fun!

In [1]:
from examples.newtons_method_optimization_example import NewtonsMethodOptimizationExample
NewtonsMethodOptimizationExample.run()

Because the derivative is quadratic, we are guaranteed to converge to a zero of the derivative, which is a stationary point of the original function $f$. Since local minima and maxima are always located at a stationary point, this is good.

A local minimum is not necessarily a global minimum. In fact, a cubic function such as we have here never has a global minimum. So our problem is a little silly! Luckily, the error functions we will try to minimize will always be positive-valued and can't get "infinitely negative."

A more significant problem can occur when there are many local minima: we might converge to the wrong one! There are a number of ways to address this problem. Maybe the simplest is to try the algorithm several times, each time using a new random starting position for $x_0$. You can pick the lowest local minima you've found. This is called *random restarts*.

Practitioners working with large neural networks have noticed that any local minimum tends to be almost as good as the global minima ([Goodfellow](http://www.deeplearningbook.org/), 8.2.2). At least in the context of large neural networks, local but non-global minima appear to be more of a theoretical problem.

We can see another problem if we started in a different position!

In [2]:
NewtonsMethodOptimizationExample.run(x_0 = -8)

Again, Newton's method has found a stationary point! However, it is a local maximum! Gah!

You will tend be attracted to a local maximum exactly whenever you are at a point where the second derivative is negative. When the second derivative is negative, the function is "decelerating." The place where the "velocity" is zero is where you flip around from increasing $f$ to decreasing it. That is a local maximum.

There are some fixes to this problem. You can review ([Goodfellow](http://www.deeplearningbook.org/), 8.6.1) to see some if you're really interested.

However, there is an even more important problem to Newton's method when the $x$ input isn't just a single real value, but a *vector* of values. That happens when you have several *predictor variables* used to estimated a *target variable*. For instance, you use both age and weight to predict lifespan.

You can still use Newton's method when $x$ is multidimensional. In that case, the analogue to the derivative is called the *gradient*: it is a vector that looks like this:

\\[
\nabla f \left( (x_1', x_2', x_3') \right) = \left(
    \frac{\partial f}{\partial x_1}(x_1'),
    \frac{\partial f}{\partial x_2}(x_2'),
    \frac{\partial f}{\partial x_3}(x_3')
\right)
\\]

The gradient is the vector of partial derivatives.

The analogue to the second derivative is called the *Hessian*. It is a *matrix* where:

\\[
H_{i, j} = \frac{\partial^2 f}{\partial x_i \partial x_j}
\\]

That is, the entry in the $i$th row and $j$th column is the *mixed partial derivative.* You calculate the mixed partial by just differentiating *twice* (just like a second derivative), except each time with respect to the variable $x_i$ and then the variable $x_j$.

So you can make Newton's method work by effectively calculating:

\\[
H^{-1} (-\nabla f)
\\]

Basically, you aren't allowed to "divide" by a matrix. But you *can* multiply by the *inverse* of a matrix. So this equation is the analogue of Newton's Method.

The problem is this. First: if $x$ is $k$-dimensional, then $H$ has $O(k^2)$ entries in it. That can be a lot of partial derivatives. Worse, the inverse of a $k$-by-$k$ matrix takes $O(k^3)$ time to calculate. When we work with neural networks, $k$ will be very large, and $O(k^3)$ is just too much computation.

There is a last problem called *numerical stability*. This basically says that if the partial derivatives are small, then when you go to invert the matrix, small numbers become big numbers. And the problem then is that any small round-off error in a small partial derivative becomes a big error in the inverted matrix.

Therefore, we will see a simplification of Newton's Method called *Gradient Descent*. Gradient descent will only use the gradient, and not use the Hessian.